In [ ]:
device = 'cuda:3'

In [ ]:
import numpy as np
import copy
import torch
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm
from include import *
import torch.optim as optim
from skimage.metrics import peak_signal_noise_ratio
PSNR= peak_signal_noise_ratio
import bm3d

In [ ]:
def crop_center(img):    
    new_width=256
    new_height=256

    width, height = img.size   # Get dimensions

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    img = img.crop((left, top, right, bottom))
    
    return np.array(img)

def add_noise(img, sigma):
    np.random.seed(1)
    noise = np.float32(np.random.normal(0, sigma/255, (256,256))) 
    noisy_img=np.clip(img+noise,0,1)
    
    return noisy_img

In [ ]:
img = Image.open('./car.png').convert('L')
img = crop_center(img)

clean_img = np.float32(img/np.max(img)) 
noisy_img = add_noise(clean_img, 30)

In [ ]:
plt.gray()
f, plots = plt.subplots(1,2)
plots[0].imshow(clean_img)
plots[1].imshow(noisy_img)

In [ ]:
noisy_img = torch.FloatTensor(noisy_img).to(device)
clean_img = torch.FloatTensor(clean_img).to(device)

noise = noisy_img - clean_img
noise = noise*torch.norm(clean_img)/torch.norm(noise)

torch.manual_seed(1)
Input = torch.rand((1,1,256,256)).to(device)

In [ ]:
model_a = Unet_Mixer().to(device)
model_b = copy.deepcopy(model_a)
model_c = copy.deepcopy(model_a)

In [ ]:
##Functions

##Loss
def mse(gt: torch.Tensor, pred:torch.Tensor)-> torch.Tensor:
    loss = torch.nn.MSELoss()
    return loss(gt,pred)

#train
def train(model, optimizer, Input, clean_img, noisy_img):
    model.train()
    
    # reset optimizer's gradient
    optimizer.zero_grad()
    
    # get the prediction
    pred = model(Input)
    #pred = torch.clamp(pred,0,1)

    mse_noisy = mse(pred, noisy_img.unsqueeze(0).unsqueeze(0))
    mse_clean = mse(pred, clean_img.unsqueeze(0).unsqueeze(0))
    
    #one step of training
    mse_noisy.backward()
    optimizer.step()

    return mse_noisy.item(), mse_clean.item()


def fit(model, optimizer, Input, img):
    model.train()
    
    # reset optimizer's gradient
    optimizer.zero_grad()
    
    # get the prediction
    pred = model(Input)
    #pred = torch.clamp(pred,0,1)

    mse_pred = mse(pred, img.unsqueeze(0).unsqueeze(0))
    
    #one step of training
    mse_pred.backward()
    optimizer.step()

    return mse_pred.item()


In [ ]:
# fitting noisy image

max_epoch = 1501
lr = 0.0005
decay_step = 500
decay_gamma = 0.5
iterations=[0,10,50,100,700,1100,1500]



optimizer = optim.Adam(model_a.parameters(), lr=lr, weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=decay_step, gamma=decay_gamma)

denoised= []
mse_noisy=[]
mse_clean=[]
err=999999


for i in range(max_epoch):
    
    a,b = train(model_a, optimizer, Input, clean_img, noisy_img)
    
    mse_noisy.append(a)
    mse_clean.append(b)
    
    scheduler.step()
    
    with torch.no_grad(): 
        if i in iterations:
            denoised.append(model_a(Input).squeeze(0).squeeze(0))

        if b<err:
            err = b
            best = model_a(Input).squeeze(0).squeeze(0)
            best_i=i

    if i%50 == 0:
        print(i, a,b, end='\r')
        


In [ ]:
#fitting clean image

optimizer = optim.Adam(model_b.parameters(), lr=lr, weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=decay_step, gamma=decay_gamma)

clean=[]


for i in range(max_epoch):
    
    err = fit(model_b, optimizer, Input, clean_img)
    clean.append(err)

    scheduler.step()
    
    if i%50 == 0:
        print(i, err, end='\r')

In [ ]:
#fitting noise

optimizer = optim.Adam(model_c.parameters(), lr=lr, weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=decay_step, gamma=decay_gamma)

noisy=[]


for i in range(max_epoch):
    
    err = fit(model_c, optimizer, Input, noise)
    noisy.append(err)

    scheduler.step()
    
    if i%50 == 0:
        print(i, err, end='\r')

In [ ]:
#mse_clean.insert(0,mse_clean[0])
plt.plot(mse_clean[150:])
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.xscale("log")

In [ ]:
noisy.insert(0,noisy[0])
clean.insert(0,clean[0])

plt.plot(clean[0:], label ='clean')
plt.plot(noisy[0:], label = 'noise')
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.legend()
plt.xscale("log")

In [ ]:
for j in range (len(iterations)):
    if best_i<iterations[j]:
        break

denoised.insert(j,best)
iterations = np.insert(iterations,j,best_i)

L=len(iterations)

In [ ]:
bm3d_img = np.clip(bm3d.bm3d(noisy_img.cpu().numpy(), 30/255),0,1).astype('float32')

In [ ]:
psnr=[]

for i in range(L):
    restored=np.clip(denoised[i].cpu().numpy(),0,1)
    psnr.append(PSNR(restored,clean_img.cpu().numpy()))

psnr.append(PSNR(bm3d_img, clean_img.cpu().numpy()))    

In [ ]:
fig, ax = plt.subplots(1, L+2,figsize=(20, 20))
for i in range(L):
    ax[i].imshow(denoised[i].cpu().numpy())
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    
    if i == j:
        ax[i].set_title('Best Iteration: '+ str(iterations[i])+'\n' +'PSNR:' +str(np.around(psnr[i],1)))
    else:
        ax[i].set_title('Iteration: '+ str(iterations[i])+'\n' +'PSNR:' +str(np.around(psnr[i],1)))
        

ax[L].imshow(bm3d_img)
ax[L].set_xticks([])
ax[L].set_yticks([])
ax[L].set_title('BM3D'+'\n' +'PSNR:' +str(np.around(psnr[L],1)))

ax[L+1].imshow(clean_img.cpu().numpy())
ax[L+1].set_xticks([])
ax[L+1].set_yticks([])
ax[L+1].set_title('Clean Image')


    